In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk 
!pip install konlpy JPype1-py3 
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

1. 이 데이터셋만 사용하여 학습(현재 베이스라인==Logistic Regression)  
2. 우리 데이터셋 안에 단어가 없다면? 말뭉치에서 유사한 단어 가져오기  
감성 말뭉치 데이터셋 사용하려면, 우리가 원하는 결과에 맞게 label을 수정하는 것이 좋을 듯

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from konlpy.tag import Okt

okt = Okt()

In [2]:
file_path ='/content/drive/MyDrive/data/centi_data/'

In [ ]:
train = pd.read_excel(file_path+'train.xlsx')

In [ ]:
train.head(5)

In [ ]:
train['감정_소분류'].unique()

array(['신이 난', '스트레스 받는', '당황', '안도', '취약한', '우울한', '구역질 나는', '좌절한',
       '눈물이 나는', '배신당한', '남의 시선을 의식하는', '죄책감의', '당혹스러운', '분노', '두려운',
       '낙담한', '회의적인', '마비된', '혼란스러운', '흥분', '한심한', '염세적인', '안달하는',
       '걱정스러운', '조심스러운', '충격 받은', '상처', '불안', '노여워하는', '기쁨', '짜증내는',
       '환멸을 느끼는', '비통한', '혐오스러운', '억울한', '만족스러운', '부끄러운', '버려진', '고립된',
       '괴로워하는', '자신하는', '외로운', '방어적인', '편안한', '질투하는', '감사하는', '초조한',
       '실망한', '슬픔', '신뢰하는', '가난한, 불우한', '후회되는', '툴툴대는', '성가신', '열등감',
       '희생된', '느긋', '악의적인'], dtype=object)

In [ ]:
unknown=['신이 난', '스트레스 받는', '당황', '구역질 나는', '좌절한', '눈물이 나는', '배신당한', '남의 시선을 의식하는', '죄책감의', '당혹스러운', '낙담한', '회의적인', '마비된',
	 '흥분', '염세적인', '안달하는', '걱정스러운', '조심스러운', '충격 받은', '노여워하는', '짜증내는', '환멸을 느끼는', '혐오스러운', '버려진', '고립된', '자신하는', '방어적인',
	 '질투하는', '신뢰하는', '가난한, 불우한', '후회되는', '툴툴대는', '성가신', '열등감', '희생된', '느긋']

In [ ]:
train['상황키워드'].unique()

array(['연애, 결혼, 출산', '건강, 죽음', '학업 및 진로', '재정', '재정, 은퇴, 노후준비', '건강',
       '학교폭력/따돌림', '대인관계', '진로, 취업, 직장', '가족관계', '직장, 업무 스트레스',
       '대인관계(부부, 자녀)'], dtype=object)

In [ ]:
y = pd.get_dummies(train['감정_소분류'])

In [ ]:
train.iloc[0][7:].dropna().tolist()
d_set = pd.DataFrame(columns=['text','label'])
for x in range(len(train)):
  train_x = train.iloc[x][7:].dropna().tolist()
  train_y = train.iloc[x]['감정_소분류']
  tmp = pd.DataFrame({'text':train_x,
                      'label':[train_y]*len(train_x)})
  d_set=d_set.append(tmp)

In [9]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import accuracy_score
import re

import gensim
from lxml import etree

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

from gensim.models.word2vec import Word2Vec

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
d_set = pd.read_csv(file_path+'train_data.csv')

In [5]:
# testing
def convert(text):
  pos = ['신이 난','기쁨','만족스러운','자신하는','편안한','감사하는','신뢰하는','안도']
  neg = ['스트레스 받는', '당황',  '취약한', '우울한', '구역질 나는', '좌절한',
		   '눈물이 나는', '배신당한', '남의 시선을 의식하는', '죄책감의', '당혹스러운', '분노', '두려운',
		   '낙담한', '회의적인', '마비된', '혼란스러운', '흥분', '한심한', '염세적인', '안달하는',
		   '걱정스러운', '조심스러운', '충격 받은', '상처', '불안', '노여워하는',
		   '짜증내는','질투하는','초조한', '실망한', '슬픔', '가난한, 불우한', '후회되는', '툴툴대는', '성가신', '열등감',
		   '희생된','느긋', '악의적인','환멸을 느끼는', '비통한', '혐오스러운', '억울한','부끄러운', '버려진', '고립된','외로운', '방어적인', 
		   '괴로워하는']
  if text in pos:
    return 1
  else:
    return -1

d_set['encoded'] = d_set['label'].apply(lambda x: convert(x))

In [6]:
okt = Okt()
def tokenizer(text):
    re.sub('[\W]',' ',text)
    result = []
    token_pos = okt.pos(text)
    for word, pos in token_pos:
        if (pos != 'KoreanParticle'):
            result.append(word)
    return result

In [7]:
x = d_set['text']
y = d_set['encoded']
# tfidf = TfidfVectorizer(max_features=1000,tokenizer=tokenizer)
# x_tdm = tfidf.fit_transform(x)

In [8]:
x = d_set['text'].apply(lambda x : tokenizer(x))
# y = d_set['encoded']
y = pd.get_dummies(d_set['label'])

word2vec 구축 시 dataset merge  
이를 하면 우리가 구축한 dataset 과 유사도 측정 가능

In [10]:
w2v =  Word2Vec(x, size=1000,
                window=10, min_count=10,workers=-1,sg=0)

In [14]:
vocab_shape = w2v.wv.vectors.shape
embedding_matrix = np.zeros((vocab_shape[0],vocab_shape[1]))
embedding_matrix = w2v.wv.vectors

In [15]:
def get_vector(word):
    if word in w2v.wv:
        return w2v.wv[word]
    else:
        return None

In [17]:
x = x.apply(lambda a : np.array([w2v.wv.index2word.index(t) for t in a if t in w2v.wv.index2word]))
max_len = max(x.apply(lambda a : len(a)).tolist())

x_train = pad_sequences(x,maxlen=max_len,padding='post')
y_train = y.to_numpy()

In [41]:
import tensorflow as tf

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_shape[0],vocab_shape[1], weights=[embedding_matrix],
                            input_length=max_len, trainable=False),
  tf.keras.layers.LSTM(units=128),
  tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 59, 1000)          10926000  
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               578048    
_________________________________________________________________
dense_7 (Dense)              (None, 58)                7482      
Total params: 11,511,530
Trainable params: 585,530
Non-trainable params: 10,926,000
_________________________________________________________________


현재 학습이 잘 안됨.  
이유  
1. embedding을 고정시켜서?  
2. lstm 구조를 바꿔야하는가?


In [42]:
history = model.fit(x_train,y_train,epochs=5,verbose=2)

Epoch 1/5
7140/7140 - 106s - loss: 4.0454 - accuracy: 0.0327
Epoch 2/5
7140/7140 - 105s - loss: 4.0443 - accuracy: 0.0332
Epoch 3/5
7140/7140 - 104s - loss: 4.0441 - accuracy: 0.0331
Epoch 4/5
7140/7140 - 104s - loss: 4.0440 - accuracy: 0.0333
Epoch 5/5
7140/7140 - 105s - loss: 4.0440 - accuracy: 0.0333


In [36]:
x_tdm

NameError: ignored

# Rogistic Regression

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_tdm,y,
                                                   test_size=0.3,
                                                   random_state=42)

In [ ]:
lr_clf = LogisticRegressionCV(max_iter=1000)
lr_clf.fit(x_train,y_train)
print(lr_clf.score(x_train,y_train))

pred =lr_clf.predict(x_test)
print(accuracy_score(y_test,pred))

0.8888680460460084
0.8882858435097241


In [ ]:
st_df = pd.DataFrame({'단어':tfidf.get_feature_names(),
                      '회귀계수':lr_clf.coef_.flat})
st_df.tail()

,단어,회귀계수
995,힘드시군요,-9.369579
996,힘든,-0.396638
997,힘들,0.291997
998,힘들고,-2.579653
999,힘들어,-5.590216


In [ ]:
st_neg = st_df[st_df['회귀계수']<0].sort_values('회귀계수')
st_neg.head()

,단어,회귀계수
461,속상하시겠어요,-15.476448
978,혼란,-13.613267
484,슬퍼,-12.903112
575,억울해,-12.781573
935,한심해,-12.416772


In [ ]:
st_pos = st_df[st_df['회귀계수']>0].sort_values('회귀계수',ascending=False)
st_pos.head()

,단어,회귀계수
190,다행,7.407279
505,신뢰,7.239126
113,기뻐,6.291304
213,덕분,6.134323
115,기쁘시군요,5.015769


In [ ]:
st_df['극성'] = np.sign(st_df['회귀계수'])

st_df.tail()
print(st_df['극성'].value_counts())

-1.0    592
 1.0    408
Name: 극성, dtype: int64


In [ ]:
st_df['극성'].sum()/st_df['극성'].abs().sum()

-0.184

In [ ]:
st_pos['단어'].tolist()

In [ ]:
st_pos[st_pos['단어']=='폭력']

,단어,회귀계수
880,폭력,0.597697
